In [ ]:
import os
import numpy as np
import pandas as pd
import pickle
from tqdm.auto import tqdm
import json
import jsonlines

current_dir = os.path.dirname(os.path.abspath(__file__))
root_path = os.path.dirname(current_dir)

In [ ]:
# # for dataset in ['IDK', 'HaluEval', 'ANAH', 'ANAH-v2']:
# for dataset in ['trivia_qa', 'pop_qa', 'nq_open']: #
#     for split in ['train', 'val', 'test']:
#         path = f"./{dataset}/sampled/{split}.csv"
#         if os.path.exists(path):
#             data = pd.read_csv(path)
            
#             # only keep id,question,answer,
#             data = data[['id', 'question', 'answer']]
#             for model in ['llama-3.1-8B-grpo']:
#                 path2 =  f"./{dataset}/{model}/{split}.csv"
#                 # make sure the directory exists
#                 os.makedirs(os.path.dirname(path2), exist_ok=True)
#                 data.to_csv(path2, index=False) 

# # Mistral-7B-Instruct-v0.3', 'Qwen2.5-7B-Instruct

# generation

In [4]:
# most likely answer
for dataset in ['trivia_qa', 'pop_qa', 'nq_open']: #
    for type in ['sentence']: #
        for model in ['Meta-Llama-3.1-8B-Instruct']: #'Mistral-7B-Instruct-v0.3', 'Qwen2.5-7B-Instruct'
            for split in ['test', 'val', 'train']: #, 
                df = pd.read_csv(f"./{dataset}/{model}/{split}.csv")
                out_dir = f"../sem_uncertainty/outputs/{dataset}/{type}/{model}/"
                path = f"{out_dir}/{split}_0.1.jsonl"
                if os.path.exists(path):
                    refusal_path = f"{out_dir}/{split}_refusal_rate.json"
                    with open(refusal_path) as f:
                        refusal = json.load(f)['refusal']
                    print("refusal rate", np.mean(refusal))
                    acc_path = f"{out_dir}/{split}_most_likely_acc.json"
                    with open(acc_path) as f:
                        accuracy = json.load(f) # dict
                    out_path = f"./{dataset}/{model}_{type}/{split}.csv"
                    os.makedirs(os.path.dirname(out_path), exist_ok=True)
                    with jsonlines.open(path) as reader:
                        data = list(reader)
                        if len(data) == len(df) == len(refusal):
                            for i, row in tqdm(df.iterrows(), total=len(df)):
                                id = str(row['id'])
                                response =  data[i]['model answers']
                                assert len(response) == 1
                                response = response[0]

                                if refusal[i] or accuracy[id]:
                                    l = 'ok'
                                else:
                                    l = 'hallucinated'
                                df.at[i, 'model_generated'] = response
                                df.at[i, f'label'] = l
                                df.at[i, 'accuracy'] = accuracy[id]
                                df.at[i, 'refusal'] = refusal[i]
                            columns_to_keep = ['id', 'question', 'model_generated', 'label', 'accuracy', 'refusal']
                            df = df[columns_to_keep]
                            df.to_csv(out_path, index=False)
                        else:
                            print(f"Length mismatch: {dataset}, {split} {len(data)} vs {len(df)} vs {len(refusal)}")
                else:
                    print(f"File not found: {path}")

refusal rate 0.054


  0%|          | 0/1000 [00:00<?, ?it/s]

refusal rate 0.048


  0%|          | 0/1000 [00:00<?, ?it/s]

refusal rate 0.0514


...

  0%|          | 0/1000 [00:00<?, ?it/s]

refusal rate 0.1


  0%|          | 0/1000 [00:00<?, ?it/s]

refusal rate 0.1069


  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
# for dataset in ["HaluEval/paired_data_rate_0.5_1000",]:
#     path = f"{root_path}/datasets/{dataset}/test.csv"
#     df = pd.read_csv(path)
#     label = df['label']
#     accuracy = [1.0 if l == 'ok' else 0.0 for l in label]
#     df['accuracy'] = accuracy
#     df.to_csv(path, index=False)

# activation

In [ ]:
import os
import shutil

for dataset in ['trivia_qa', 'nq_open', 'pop_qa']:
    for model in ['Mistral-7B-Instruct-v0.3', 'Qwen2.5-7B-Instruct']:
        target_directory = f"{root_path}/datasets/{dataset}/{model}/only_question_last_activations" 
        link_name = f"{root_path}/datasets/{dataset}/{model}_sentence/only_question_last_activations"
        os.remove(link_name)
        os.symlink(target_directory, link_name)
    # target_directory = f"{root_path}/datasets/{dataset}/Meta-Llama-3.1-8B-Instruct/only_question_last_activations" 
    # link_name = f"{root_path}/datasets/{dataset}/Meta-Llama-3.1-8B-Instruct_sentence/only_question_last_activations"
    # os.remove(link_name)
    # os.symlink(target_directory, link_name)

# verbal_uncertainty

In [ ]:
for dataset in ['trivia_qa', 'nq_open', 'pop_qa']: #
    for split in ['test', 'val']: # 'train', 
        for model in ['llama-3.1-8B-grpo']: #'Qwen2.5-7B-Instruct', 'Mistral-7B-Instruct-v0.3' Meta-Llama-3.1-8B-Instruct
            input_path = f"{dataset}/{model}/{split}.csv"
            df = pd.read_csv(input_path)
            path = f"../verbal_uncertainty/outputs_10/{model}_{dataset}_{split}_uncertainty_1.0_lu-llm-judge.json"
            if os.path.exists(path):
                with open(path, "rb") as reader:
                    data = json.load(reader)
                    # assert len(data) == len(df)
                    if len(data) == len(df):
                        for i, row in tqdm(df.iterrows(), total=len(df)):
                            assert len(data[i]) == 10
                            verbal_uncertainty = np.mean([x for x in data[i] if x !=-1])
                            df.at[i, 'verbal_uncertainty'] = verbal_uncertainty

                        df.to_csv(input_path, index=False)
                    else:
                        print(f"Length mismatch: {dataset}, {split} {len(data)} vs {len(df)}")
            else:
                print(f"File not found: {path}")

100%|██████████| 1000/1000 [00:00<00:00, 19408.10it/s]


In [ ]:
# trivia_qa, train 9300 vs 10000
for dataset in ['IDK']: #
    for split in ['test']: #
        input_path = f"{root_path}/datasets/{dataset}/sampled/{split}.csv"
        df = pd.read_csv(input_path)
        path = f"{root_path}/verbal_uncertainty/outputs/Meta-Llama-3.1-8B-Instruct_{dataset}_{split}_uncertainty_1.0_lu-llm-judge.p"
        if os.path.exists(path):
            with open(path, "rb") as reader:
                data = pickle.load(reader)
                # assert len(data) == len(df)
                if len(data) == len(df):
                    for i, row in tqdm(df.iterrows(), total=len(df)):
                        verbal_uncertainty = np.mean(data[i])
                        df.at[i, 'verbal_uncertainty'] = verbal_uncertainty

                    df.to_csv(input_path, index=False)
                else:
                    print(f"Length mismatch: {dataset}, {split} {len(data)} vs {len(df)}")
        else:
            print(f"File not found: {path}")

100%|██████████| 500/500 [00:00<00:00, 18377.37it/s]


In [ ]:

# for dataset in ['trivia_qa', 'nq_open', 'pop_qa']: #
#     for split in ['train', 'val', 'test']: #
#         input_path = f"{root_path}/datasets/{dataset}/sampled/{split}.csv"
#         df = pd.read_csv(input_path)
#         path = f"{root_path}/sem_uncertainty/outputs/{dataset}/sentence/{split}_verbal_uncertainty.pkl"
#         if os.path.exists(path):
#             with open(path, "rb") as reader:
#                 data = pickle.load(reader)
#                 # assert len(data) == len(df)
#                 if len(data) == len(df):
#                     for i, row in tqdm(df.iterrows(), total=len(df)):
#                         verbal_uncertainty = np.mean(data[row['id']])
#                         df.at[i, 'natural_verbal_uncertainty'] = verbal_uncertainty

#                     df.to_csv(input_path, index=False)
#                 else:
#                     print(f"Length mismatch: {dataset}, {split} {len(data)} vs {len(df)}")
#         else:
#             print(f"File not found: {path}")

In [ ]:
# use_predicted_uncertainty



# semantic entropy

In [ ]:

for dataset in ['trivia_qa', 'nq_open', 'pop_qa']: #trivia_qa, 'nq_open', 'pop_qa'
    for type in ['sentence']: #'no_refuse_word',
        for model in ['Mistral-7B-Instruct-v0.3', 'Qwen2.5-7B-Instruct']:
            for split in ['train', 'val', 'test']: 
                out_path = input_path = f"./{dataset}/{model}/{split}.csv"
                df = pd.read_csv(input_path)
                path = f"{root_path}/sem_uncertainty/outputs/{dataset}/{type}/{model}/{split}_semantic_entropy.pkl"
                if os.path.exists(path):
                    with open(path, "rb") as reader:
                        data = pickle.load(reader)
                        data = data['uncertainty_measures']['cluster_assignment_entropy']
                        # assert len(data) == len(df)
                        if len(data) == len(df):
                            for i, row in tqdm(df.iterrows(), total=len(df)):
                                df.at[i, f'{type}_semantic_entropy'] = data[i]
                            df.to_csv(out_path, index=False)
                        else:
                            print(f"Length mismatch: {dataset}{model}, {split} {len(data)} vs {len(df)}")
                else:
                    print(f"File not found: {path}")

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

# eigen

In [ ]:
for dataset in ['IDK']:
    for type in ['no_refuse_sentence']:
        for split in ['test']:
            out_path = f"{root_path}/datasets/{dataset}/sampled/{split}.csv"
            df = pd.read_csv(out_path)
            path = f"{root_path}/sem_uncertainty/outputs/{dataset}/{type}/{split}_eigen.pkl"
            if os.path.exists(path):
                with open(path, "rb") as reader:
                    all_results = pickle.load(reader)
                    assert len(all_results) == len(df)

                    if len(all_results) == len(df):
                        for i, row in tqdm(df.iterrows(), total=len(df)):
                            id = row['id']
                            if dataset == 'IDK':
                                id = str(id)
                            df.at[i, f'{type}_eigen'] = all_results[id]["eigenIndicator"]

                        df.to_csv(out_path, index=False)
                    else:
                        print(f"Length mismatch: {dataset}, {split} {len(data)} vs {len(df)}")
            else:
                print(f"File not found: {path}")

100%|██████████| 500/500 [00:00<00:00, 17627.13it/s]


# refusal

In [ ]:

import json

for dataset in ['trivia_qa', 'nq_open', 'pop_qa']:
    for split in ['train', 'val', 'test']:
        out_path = f"{root_path}/datasets/{dataset}/sampled/{split}.csv"
        df = pd.read_csv(out_path)
        data_path = "{root_path}/sem_uncertainty/outputs/pop_qa/word/test_refusal_rate.json"
        if os.path.exists(data_path):
            with open(data_path) as reader:
                data = json.load(reader)
                all_results = data["refusal"]
                assert len(all_results) == len(df)

                if len(all_results) == len(df):
                    for i, row in tqdm(df.iterrows(), total=len(df)):
                        id = row['id']
                        df.at[i, f'if_refusal'] = all_results[id]
                    df.to_csv(out_path, index=False)
                else:
                    print(f"Length mismatch: {dataset}, {split} {len(data)} vs {len(df)}")
        else:
            print(f"File not found: {path}")

dict_keys(['semantic_ids', 'validation_is_false', 'validation_unanswerable', 'uncertainty_measures', 'alt_validation_accuracies_mean', 'alt_validation_is_false'])

# predicted test data

In [ ]:
# # os.walk get fall files in datastes named train.csv

# import os
# import pandas as pd
# import numpy as np

# for root, dirs, files in os.walk("{root_path}/datasets/"):
#     for file in files:
#         if 'trivia_qa' not in root and 'nq_open' not in root and 'pop_qa' not in root:
#             if file in ["test.csv", 'val.csv', 'train.csv']:
#                 path = os.path.join(root, file)
#                 df = pd.read_csv(path)
#                 # rename the column answer to model_generated
#                 df = df.rename(columns={"answer": "model_generated"})
#                 df.to_csv(path, index=False)